In [ ]:
import torch
import torch.onnx

# 1. Импортируй свою модель
from your_model_file import YourModelClass  # Заменить на свой путь и класс

# 2. Создание и загрузка модели
model = YourModelClass()  # или передай аргументы, если нужно
checkpoint = torch.load('my_checkpoint.pth', map_location='cpu')
model.load_state_dict(checkpoint)
model.eval()

# 3. Пример входных данных
dummy_input = torch.randn(1, 3, 224, 224)  # Заменить на подходящий размер

# 4. Экспорт в ONNX
torch.onnx.export(
    model,                      # модель
    dummy_input,                # пример входа
    "exported_model.onnx",      # имя выходного файла
    export_params=True,
    opset_version=11,           # можешь поменять на 12 или выше, если нужно
    do_constant_folding=True,   # оптимизация
    input_names=['input'],      # необязательно
    output_names=['output'],    # необязательно
    dynamic_axes={
        'input': {0: 'batch_size'},  # позволяет менять размер батча
        'output': {0: 'batch_size'}
    }
)
